In [1]:
import os
from dotenv import load_dotenv
from snowflake.snowpark import Session

ModuleNotFoundError: No module named 'snowflake.snowpark'

In [3]:
load_dotenv()

True

In [4]:
ACCOUNT = os.getenv('ACCOUNT')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')
ROLE = os.getenv('ROLE')
WAREHOUSE = os.getenv('WAREHOUSE')
DATABASE = os.getenv('DATABASE')
SCHEMA = os.getenv('SCHEMA')

print(ACCOUNT, USER, ROLE, WAREHOUSE, DATABASE, SCHEMA)

hum.us-east-2.aws erictria UVARND UVARND_WH CORE CLIENT


In [5]:
connection_parameters = {
    'account': os.getenv('ACCOUNT'),
    #'user': USER,
    'user': 'ericatuva',
    'password': os.getenv('PASSWORD'),
    'role': os.getenv('ROLE'),
    'warehouse': os.getenv('WAREHOUSE'),
    'database': os.getenv('DATABASE'),
    'schema': os.getenv('SCHEMA')
}

In [ ]:
test_session = Session.builder.configs(connection_parameters).create()

In [ ]:
test_query = """
SELECT * 
FROM event
WHERE day = DATE('2022-06-01')
LIMIT 10
"""